In [1]:
!pip install yfinance


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# POC - Descobrir os componentes financeiros com forte correlação com "Dividend"

In [39]:
import pandas as pd
import yfinance as yf

# Escolha do ativo, por exemplo, 'PETR4.SA' para Petrobras
ticker = 'PETR4.SA'

# Download dos dados do ativo
stock = yf.Ticker(ticker)

# Coletando dados financeiros e históricos de dividendos
dividends = stock.dividends
financials = stock.financials.transpose()
balance_sheet = stock.balance_sheet.transpose()
cash_flow = stock.cashflow.transpose()

# Agrupar dividendos por ano
dividends = dividends.to_frame().reset_index()
grouped_dividends = dividends.groupby(pd.Grouper(key='Date', freq='Y')).sum()
grouped_dividends.index = grouped_dividends.index.tz_localize(None)

# Consolidando todos os dados em um único DataFrame
data_frames = [financials, balance_sheet, cash_flow]
df = pd.concat(data_frames, axis=1)

# Tratar dados ausentes
df.fillna(inplace=True, value=0)

# Juntar "grouped_dividends" com "df" apenas onde "df" possui a data (index)
df = df.merge(grouped_dividends, left_index=True, right_index=True)

df

,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,...,Depreciation And Amortization,Amortization Cash Flow,Amortization Of Intangibles,Depreciation,Operating Gains Losses,Pension And Employee Benefit Expense,Earnings Losses From Equity Investments,Net Foreign Currency Exchange Gain Loss,Net Income From Continuing Operations,Dividends
2023-12-31,-2.734200e+08,0.294000,5.322600e+10,-9.300000e+08,-9.300000e+08,2.488400e+10,1.328000e+10,3.588700e+10,5.229600e+10,3.901600e+10,...,1.328000e+10,104000000.0,104000000.0,1.317600e+10,3.049000e+09,1.542000e+09,3.040000e+08,2.498000e+09,2.499500e+10,7.344291
2022-12-31,6.303820e+08,0.313000,6.804000e+10,2.014000e+09,2.014000e+09,3.662300e+10,1.321800e+10,4.716200e+10,7.005400e+10,5.683600e+10,...,1.321800e+10,77000000.0,77000000.0,1.314100e+10,4.390000e+09,1.228000e+09,-2.510000e+08,4.557000e+09,3.675500e+10,15.102379
2021-12-31,-3.972818e+08,0.291904,4.625800e+10,-1.361000e+09,-1.361000e+09,1.987500e+10,1.169500e+10,3.216500e+10,4.489700e+10,3.320200e+10,...,1.169500e+10,60000000.0,60000000.0,1.163500e+10,9.386000e+09,2.098000e+09,-1.607000e+09,1.079500e+10,1.998600e+10,5.653232
2020-12-31,-4.959920e+09,0.340000,3.157800e+10,-1.458800e+10,-1.458800e+10,1.141000e+09,1.144500e+10,1.840300e+10,1.699000e+10,5.545000e+09,...,1.144500e+10,66000000.0,66000000.0,1.137900e+10,1.029600e+10,-1.001000e+09,6.590000e+08,1.109400e+10,9.480000e+08,0.000461
2019-12-31,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.942361


In [40]:
correlation_matrix = df.corr()
correlation_matrix

,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,...,Depreciation And Amortization,Amortization Cash Flow,Amortization Of Intangibles,Depreciation,Operating Gains Losses,Pension And Employee Benefit Expense,Earnings Losses From Equity Investments,Net Foreign Currency Exchange Gain Loss,Net Income From Continuing Operations,Dividends
Tax Effect Of Unusual Items,1.000000,-0.354871,0.225622,0.999791,0.999791,0.600973,-0.140021,0.303434,0.443105,0.556589,...,-0.140021,-0.069970,-0.069970,-0.140445,-0.658347,0.739806,-0.482524,-0.629457,0.604701,0.630905
Tax Rate For Calcs,-0.354871,1.000000,0.819777,-0.355527,-0.355527,0.512450,0.970389,0.774109,0.670132,0.562721,...,0.970389,0.867582,0.867582,0.970699,0.736551,0.215986,-0.025653,0.686810,0.508995,0.396367
Normalized EBITDA,0.225622,0.819777,1.000000,0.227692,0.227692,0.911382,0.916825,0.996487,0.973316,0.933583,...,0.916825,0.862390,0.862390,0.916825,0.312328,0.594271,-0.209429,0.272426,0.909650,0.837803
Total Unusual Items,0.999791,-0.355527,0.227692,1.000000,1.000000,0.603936,-0.141453,0.305323,0.445057,0.559234,...,-0.141453,-0.072455,-0.072455,-0.141869,-0.663087,0.730595,-0.473175,-0.634427,0.607612,0.638472
Total Unusual Items Excluding Goodwill,0.999791,-0.355527,0.227692,1.000000,1.000000,0.603936,-0.141453,0.305323,0.445057,0.559234,...,-0.141453,-0.072455,-0.072455,-0.141869,-0.663087,0.730595,-0.473175,-0.634427,0.607612,0.638472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pension And Employee Benefit Expense,0.739806,0.215986,0.594271,0.730595,0.730595,0.758965,0.404556,0.648078,0.718697,0.753663,...,0.404556,0.402678,0.402678,0.404404,-0.068636,1.000000,-0.729734,-0.055829,0.761672,0.635046
Earnings Losses From Equity Investments,-0.482524,-0.025653,-0.209429,-0.473175,-0.473175,-0.329633,-0.084528,-0.263357,-0.304111,-0.339361,...,-0.084528,0.107839,0.107839,-0.085813,-0.239860,-0.729734,1.000000,-0.303357,-0.332359,-0.259447
Net Foreign Currency Exchange Gain Loss,-0.629457,0.686810,0.272426,-0.634427,-0.634427,-0.068326,0.530510,0.228507,0.101031,-0.005854,...,0.530510,0.280325,0.280325,0.532010,0.995953,-0.055829,-0.303357,1.000000,-0.071004,-0.165565
Net Income From Continuing Operations,0.604701,0.508995,0.909650,0.607612,0.607612,0.999987,0.677263,0.938834,0.979756,0.997739,...,0.677263,0.664582,0.664582,0.677075,-0.049193,0.761672,-0.332359,-0.071004,1.000000,0.967158


In [48]:
# Mostrar os atributos com correlação significativa com "Dividend"
dividend_corr = correlation_matrix['Dividends'].dropna()
dividend_features = dividend_corr[(abs(dividend_corr) >= 0.7) & (abs(dividend_corr) < 1.0)]
dividend_features

Normalized EBITDA                                             0.837803
Net Income From Continuing Operation Net Minority Interest    0.967344
Reconciled Cost Of Revenue                                    0.870373
EBITDA                                                        0.920954
EBIT                                                          0.957789
Normalized Income                                             0.916749
Net Income From Continuing And Discontinued Operation         0.967344
Total Expenses                                                0.808004
Diluted EPS                                                   0.967533
Basic EPS                                                     0.967533
Diluted NI Availto Com Stockholders                           0.967344
Net Income Common Stockholders                                0.967344
Net Income                                                    0.967344
Minority Interests                                           -0.759764
Net In

# Algoritmo para descobrir os componentes financeiros com maior correlação com "Dividend"

In [53]:
import yfinance as yf
import pandas as pd

# Lista de ativos
ativos = {
    'VAMO3': 'VAMO3.SA',
    'VBBR3': 'VBBR3.SA',
    'VIVA3': 'VIVA3.SA',
    'VIVT3': 'VIVT3.SA',
    'VLID3': 'VLID3.SA',
    'VULC3': 'VULC3.SA',
    'WEGE3': 'WEGE3.SA',
    'WIZC3': 'WIZC3.SA',
    'YDUQ3': 'YDUQ3.SA',
    'ZAMP3': 'ZAMP3.SA'
}

# Dicionário para armazenar correlações
correlations = []

# Função para coletar dados e calcular correlação
def get_correlations(ticker):
    stock = yf.Ticker(ticker)

    # Coletando dados financeiros e históricos de dividendos
    dividends = stock.dividends
    financials = stock.financials.transpose()
    balance_sheet = stock.balance_sheet.transpose()
    cash_flow = stock.cashflow.transpose()

    # Agrupar dividendos por ano
    dividends = dividends.to_frame().reset_index()
    grouped_dividends = dividends.groupby(pd.Grouper(key='Date', freq='Y')).sum()
    grouped_dividends.index = grouped_dividends.index.tz_localize(None)

    # Consolidando todos os dados em um único DataFrame
    data_frames = [financials, balance_sheet, cash_flow]
    df = pd.concat(data_frames, axis=1)

    # Tratar dados ausentes
    df.fillna(inplace=True, value=0)

    # Juntar "grouped_dividends" com "df" apenas onde "df" possui a data (index)
    df = df.merge(grouped_dividends, left_index=True, right_index=True)

    # Calculando a correlação
    correlation_matrix = df.corr()
    
    # Retornar apenas a coluna 'Dividends' da correlação
    if 'Dividends' in correlation_matrix:
        dividend_corr = correlation_matrix['Dividends'].dropna()
        return dividend_corr
    else:
        return None

# Coletar correlações para todos os ativos
for nome, ticker in ativos.items():
    corr = get_correlations(ticker)
    if corr is not None:
        correlations.append(corr)

# Convertendo a lista de correlações em um DataFrame
correlation_df = pd.DataFrame(correlations)

# Calculando a média das correlações
mean_correlation = correlation_df.median().dropna()

# Filtrando os componentes financeiros com forte correlação
strong_correlations = mean_correlation[(abs(mean_correlation) >= 0.7) & (abs(mean_correlation) < 1.0)]

In [54]:
strong_correlations

Finished Goods                                       0.839278
Depreciation Income Statement                       -0.703512
Pension And Employee Benefit Expense                 0.785728
Line Of Credit                                      -0.762415
Amortization Cash Flow                              -0.736729
Investments In Other Ventures Under Equity Method    0.742601
Sale Of Intangibles                                  0.894391
dtype: float64

In [64]:
import pickle

with open("data/dividend_features.pkl", "wb") as file:
    pickle.dump(strong_correlations.index.to_list(), file)